In [8]:
# %load_ext autotime
# !pip install autotime
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import warnings
import pandas as pd
import numpy as np
import time
import esm
import biotite.structure.io as bsio
from proteinttt.models.esmfold import ESMFoldTTT, DEFAULT_ESMFOLD_TTT_CFG
from proteinttt.utils.structure import calculate_tm_score, lddt_score
import torch
import argparse
import os


pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
base_path = Path("/scratch/project/open-35-8/pimenol1/ProteinTTT/ProteinTTT/data/bfvd/")
SUMMARY_PATH = base_path / 'proteinttt_results.tsv'
MSA_PATH = Path("/scratch/project/open-35-8/antonb/bfvd/bfvd_msa")
SUBSET_PATH = base_path / 'proteinttt_msa_testset.tsv'

# Choosing data

In [11]:
df = pd.read_csv(base_path / 'logs_df' / 'A0A0H4SQ70_log.tsv', sep="\t")
log = pd.read_csv(base_path / 'logs' / 'A0A0H4SQ70_log.tsv', sep="\t")

In [12]:
log

,ttt_step_data,df
0,"defaultdict(<class 'dict'>, {0: {'eval_step_preds': {'pdb': 'PARENT N/A\nATOM 1 N MET A 1 -6.334 -13.737 14.853 1.00 30.43 N \nATOM 2 CA MET A 1 -5.347 -14.366 13.981 1.00 40.25 C \nATOM 3 C MET A 1 -5.152 -13.556 12.704 1.00 31.69 C \nATOM 4 CB MET A 1 -5.768 -15.795 13.634 1.00 30.67 C \nATOM 5 O MET A 1 -6.104 -13.335 11.953 1.00 31.98 O \nATOM 6 CG MET A 1 -4.754 -16.851 14.044 1.00 30.73 C \nATOM 7 SD MET A 1 -5.402 -18.557 13.861 1.00 18.27 S \nATOM 8 CE MET A 1 -4.304 -19.179 12.557 1.00 18.19 C \nATOM 9 N GLN A 2 -4.185 -12.569 12.812 1.00 53.99 N \nATOM 10 CA GLN A 2 -3.840 -11.817 11.610 1.00 55.16 C \nATOM 11 C GLN A 2 -3.514 -12.753 10.450 1.00 55.60 C \nATOM 12 CB GLN ...",step accumulated_step loss ... score_seq_time eval_step_time plddt\n0 0 0 NaN ... 0.0 1.979763 85.136031\n1 1 4 1.396484 ... 0.0 1.963782 84.821798\n2 2 8 1.220703 ... 0.0 1.963898 85.219783\n3 3 12 1.214844 ... 0.0 1.965569 85.962069\n4 4 16 0.666016 ... 0.0 1.964066 86.218772\n5 5 20 0.564453 ... 0.0 1.964790 86.328141\n6 6 24 0.369873 ... 0.0 1.962968 86.517463\n7 7 28 0.220459 ... 0.0 1.964930 86.860556\n8 8 32 0.119141 ... 0.0 1.965095 86.353019\n9 9 36 0.122437 ... 0.0 1.965097 86.658853\n10 10 40 0.067566 ... 0.0 ...


In [3]:
from Bio import SeqIO


def check_a3m_lengths(filepath):
    lengths = {}
    try:
        for record in SeqIO.parse(filepath, format="fasta"):
            # DELETE all lowercase letters from the sequence
            for char in record.seq:
                if char.islower():
                    record.seq = record.seq.replace(char, "")
            lengths[record.id] = len(record.seq)

        first_record_id = list(lengths.keys())[0]
        reference_length = lengths[first_record_id]

        print(f"Reference sequence: '{first_record_id}' (Length: {reference_length})")

        is_consistent = True
        for seq_id, length in lengths.items():
            if length != reference_length:
                print(f"MISMATCH: '{seq_id}' has length {length}")
                is_consistent = False

        if is_consistent:
            print(f"All {len(lengths)} sequences have the same length: {reference_length}")

    except Exception as e:
        print(f"An error occurred: {e}")

file_to_check = "/scratch/project/open-35-8/antonb/bfvd/bfvd_msa/A0A1M7XUY2.a3m"
check_a3m_lengths(file_to_check)

Reference sequence: 'UniRef100_A0A1M7XUY2' (Length: 437)
All 2527 sequences have the same length: 437
